In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/12/04 03:28:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT   
    -- Reveneue grouping 
    date_trunc("hour", tpep_pickup_datetime)  AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM 
    yellow
WHERE tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 
    1, 2
```

In [3]:
rdd = df_green  \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime
from collections import namedtuple
from pyspark.sql import types
import pandas as pd

In [5]:
start = datetime(year = 2020, month=1, day=1)

def filteroutliers(row):
    return row.lpep_pickup_datetime>=start

In [6]:
rows = rdd.take(1)
row = rows[0]

In [7]:
row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)

datetime.datetime(2020, 1, 19, 1, 0)

In [8]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 19, 1, 55, 32), PULocationID=181, total_amount=25.05)

In [9]:
def prep_for_grouping(row):
    hour =row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID 
    key = (hour, zone)

    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [10]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [11]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count' ])

In [12]:
def unwrap(row):
    return RevenueRow(
        hour = row[0][0], 
        zone = row[0][1],
        revenue = row[1][0], 
        count = row[1][1])

In [13]:
results_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True), 
    types.StructField('zone', types.IntegerType(), True), 
    types.StructField('revenue', types.DoubleType(), True), 
    types.StructField('count', types.IntegerType(), True)])

In [14]:
df_results = rdd \
    .filter(filteroutliers)\
    .map(prep_for_grouping)\
    .reduceByKey(calculate_revenue)\
    .map(unwrap)\
    .toDF(results_schema)

In [ ]:
df_results.write.parquet('temp/green_revenue')

In [15]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green\
    .select(columns)\
    .rdd

In [16]:
rows=duration_rdd.take(10)

In [17]:
df =pd.DataFrame(rows, columns = columns)

In [18]:
def model_prediction(df):
    #y_pred = model.predict(df) # define model first
    y_pred = df.trip_distance*5
    return y_pred

In [19]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns = columns)
    predictions = model_prediction(df)
    df['predicted_duration'] = predictions
    
    for row in df.itertuples():
        yield row

In [20]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF()\
    .drop('index')

In [22]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|             31.25|
|             12.75|
|             11.05|
|54.349999999999994|
|             91.75|
|               3.1|
|11.899999999999999|
|              4.45|
|              4.85|
|              15.1|
|             30.65|
|              3.55|
|               6.7|
|               3.5|
|              8.95|
|             39.95|
|              14.2|
|              30.9|
|               5.5|
|               8.8|
+------------------+
only showing top 20 rows

